In [ ]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install xlrd

In [ ]:
# Unit of analysis: census tract
# DV: nLoans1, amtLoans1, etc. Also: (overall minus "credit card loans")
#### where credit card loans" are <$100k to businesses with <$10k budget

# IV: [population by race; owner-occupied units; MFI; hs grad pct; 
### female hoh pct; unemployment pct; poverty pct; median home value;
### median home year built

In [127]:
import numpy as np
import pandas
import os
from IPython.display import display, HTML
import time
import re
print('packages imported...')

pandas.set_option('display.max_colwidth', -1)
basedir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/datasets/acs5/'
outdir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/datasets/acs5/'

print('directories set...')
print('Done!')

packages imported...
directories set...
Done!


# Read all metadata

In [8]:
metadata_df = pandas.read_csv(basedir+'metadata.csv', low_memory=False, index_col='Unnamed: 0')
metadata_df = metadata_df.reset_index(drop=True)
metadata_df.index.name = 'varnum'
metadata_df

,variable,description,sequence_number
varnum,,,
0,FILEID,FILEID,0
1,FILETYPE,FILETYPE,0
2,STUSAB,STUSAB,0
3,CHARITER,CHARITER,0
4,SEQUENCE,SEQUENCE,0
5,LOGRECNO,LOGRECNO,0
6,B00001_001,UNWEIGHTED SAMPLE COUNT OF THE POPULATION% Total population%Total,1
7,B00002_001,UNWEIGHTED SAMPLE HOUSING UNITS% Housing units%Total,1
8,B01001_001,SEX BY AGE% Total population%Total,2


# Select race-related indpendent variables

In [80]:
# IV: [population by race; owner-occupied units; MFI; hs grad pct; 
### female hoh pct; unemployment pct; poverty pct; median home value;
### median home year built

xdf = metadata_df[metadata_df['description'].apply(lambda x: 'race' in x.lower())]
xdf.loc[:, 'description'] = xdf['description'].astype('str')

thehtml = '<table>'
thehtml += '<tr><th>varnum</th><th>variable</th><th>description</th><th>sequence</th></tr>'
for ix, thisrow in xdf[(xdf['sequence_number'] == 4) | (xdf['sequence_number'] == 5)].iterrows():
    thehtml += '<tr>'
    thehtml += '<td>'+str(ix)+'</td>'
    thehtml += '<td>'+thisrow['variable']+'</td>'
    thisdesc = thisrow['description'].replace('%','<br />')
    thehtml += '<td>'+thisdesc+'</td>'
    thehtml += '<td>'+str(thisrow['sequence_number'])+'</td>'
    thehtml += '</tr>'
thehtml += '</table>'

display(HTML(thehtml))

ind_vars = ['B02001_001', 'B02001_002', 'B02001_003', 'B03002_001', 'B03002_012']



varnum,variable,description,sequence
367,B02001_001,RACE Total populationTotal,4
368,B02001_002,RACETotal populationTotalWhite alone,4
369,B02001_003,RACETotal populationTotalBlack or African American alone,4
370,B02001_004,RACETotal populationTotalAmerican Indian and Alaska Native alone,4
371,B02001_005,RACETotal populationTotalAsian alone,4
372,B02001_006,RACETotal populationTotalNative Hawaiian and Other Pacific Islander alone,4
373,B02001_007,RACETotal populationTotalSome other race alone,4
374,B02001_008,RACETotal populationTotalTwo or more races,4
375,B02001_009,RACETotal populationTotalTwo or more racesTwo races including Some other race,4
376,B02001_010,RACETotal populationTotalTwo or more racesTwo races excluding Some other race and three or more races,4


# Select income-related IVs

In [67]:
# IV: [population by race; owner-occupied units; MFI; hs grad pct; 
### female hoh pct; unemployment pct; poverty pct; median home value;
### median home year built

xdf = metadata_df[metadata_df['sequence_number'].apply(lambda x: (x >= 59) & (x <= 72))]
xdf = xdf[xdf['description'].apply(lambda x: ('mfi' in x.lower()) | ('median family income' in x.lower()))]
xdf.loc[:, 'description'] = xdf['description'].astype('str')

thehtml = '<table>'
thehtml += '<tr><th>varnum</th><th>variable</th><th>description</th><th>sequence</th></tr>'
for ix, thisrow in xdf.iterrows():
    thehtml += '<tr>'
    thehtml += '<td>'+str(ix)+'</td>'
    thehtml += '<td>'+thisrow['variable']+'</td>'
    thisdesc = thisrow['description'].replace('%','<br /><br />')
    thehtml += '<td>'+thisdesc+'</td>'
    thehtml += '<td>'+str(thisrow['sequence_number'])+'</td>'
    thehtml += '</tr>'
thehtml += '</table>'

display(HTML(thehtml))

ind_vars += ['B19113_001', 'B19113A_001', 'B19113B_001']



varnum,variable,description,sequence
14911,B19113_001,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) FamiliesMedian family income in the past 12 months (in 2017 inflation-adjusted dollars),64
14912,B19113A_001,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) (WHITE ALONE HOUSEHOLDER) Families with a householder who is White aloneMedian family income in the past 12 months (in 2017 inflation-adjusted dollars),64
14913,B19113B_001,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) (BLACK OR AFRICAN AMERICAN ALONE HOUSEHOLDER) Families with a householder who is Black or African American aloneMedian family income in the past 12 months (in 2017 inflation-adjusted dollars),64
14914,B19113C_001,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) (AMERICAN INDIAN AND ALASKA NATIVE ALONE HOUSEHOLDER) Families with a householder who is American Indian and Alaska Native aloneMedian family income in the past 12 months (in 2017 inflation-adjusted dollars),64
14915,B19113D_001,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) (ASIAN ALONE HOUSEHOLDER) Families with a householder who is Asian aloneMedian family income in the past 12 months (in 2017 inflation-adjusted dollars),64
14916,B19113E_001,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE HOUSEHOLDER) Families with a householder who is Native Hawaiian and Other Pacific Islander aloneMedian family income in the past 12 months (in 2017 inflation-adjusted dollars),64
14917,B19113F_001,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) (SOME OTHER RACE ALONE HOUSEHOLDER) Families with a householder who is Some other race aloneMedian family income in the past 12 months (in 2017 inflation-adjusted dollars),64
14918,B19113G_001,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) (TWO OR MORE RACES) Families with a householder who is Two or more racesMedian family income in the past 12 months (in 2017 inflation-adjusted dollars),64
14919,B19113H_001,"MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) (WHITE ALONE, NOT HISPANIC OR LATINO HOUSEHOLDER) Families with a householder who is White alone, not Hispanic or LatinoMedian family income in the past 12 months (in 2017 inflation-adjusted dollars)",64
14920,B19113I_001,MEDIAN FAMILY INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS) (HISPANIC OR LATINO HOUSEHOLDER) Families with a householder who is Hispanic or LatinoMedian family income in the past 12 months (in 2017 inflation-adjusted dollars),64


#  Select female householder IVs

In [65]:
# IV: [population by race; owner-occupied units; MFI; hs grad pct; 
### female hoh pct; unemployment pct; poverty pct; median home value;
### median home year built

xdf = metadata_df[metadata_df['sequence_number'].apply(lambda x: (x >= 34) & (x <= 40))]
xdf = xdf[xdf['description'].apply(lambda x: ('female' in x.lower()) & ('householder' in x.lower()))]
xdf.loc[:, 'description'] = xdf['description'].astype('str')

thehtml = '<table>'
thehtml += '<tr><th>varnum</th><th>variable</th><th>description</th><th>sequence</th></tr>'
for ix, thisrow in xdf.iterrows():
    thehtml += '<tr>'
    thehtml += '<td>'+str(ix)+'</td>'
    thehtml += '<td>'+thisrow['variable']+'</td>'
    thisdesc = thisrow['description'].replace('%','<br /><br />')
    thehtml += '<td>'+thisdesc+'</td>'
    thehtml += '<td>'+str(thisrow['sequence_number'])+'</td>'
    thehtml += '</tr>'
thehtml += '</table>'

display(HTML(thehtml))

ind_vars += ['B11001_001', 'B11001_006']




varnum,variable,description,sequence
7498,B09002_015,OWN CHILDREN UNDER 18 YEARS BY FAMILY TYPE AND AGEOwn children under 18 yearsTotalIn other familiesFemale householder no husband present,34
7499,B09002_016,OWN CHILDREN UNDER 18 YEARS BY FAMILY TYPE AND AGEOwn children under 18 yearsTotalIn other familiesFemale householder no husband presentUnder 3 years,34
7500,B09002_017,OWN CHILDREN UNDER 18 YEARS BY FAMILY TYPE AND AGEOwn children under 18 yearsTotalIn other familiesFemale householder no husband present3 and 4 years,34
7501,B09002_018,OWN CHILDREN UNDER 18 YEARS BY FAMILY TYPE AND AGEOwn children under 18 yearsTotalIn other familiesFemale householder no husband present5 years,34
7502,B09002_019,OWN CHILDREN UNDER 18 YEARS BY FAMILY TYPE AND AGEOwn children under 18 yearsTotalIn other familiesFemale householder no husband present6 to 11 years,34
7503,B09002_020,OWN CHILDREN UNDER 18 YEARS BY FAMILY TYPE AND AGEOwn children under 18 yearsTotalIn other familiesFemale householder no husband present12 to 17 years,34
7508,B09005_005,"HOUSEHOLD TYPE FOR CHILDREN UNDER 18 YEARS IN HOUSEHOLDS (EXCLUDING HOUSEHOLDERS, SPOUSES, AND UNMARRIED PARTNERS)Population under 18 years in households (excluding householders, spouses, and unmarried partners)TotalIn family householdsIn female householder no husband present family",34
7514,B09008_005,"PRESENCE OF UNMARRIED PARTNER OF HOUSEHOLDER BY HOUSEHOLD TYPE FOR CHILDREN UNDER 18 YEARS IN HOUSEHOLDSPopulation under 18 years in households (excluding householders, spouses, and unmarried partners)TotalUnmarried partner of householder presentIn family householdsIn female householder no husband present family",34
7520,B09008_011,"PRESENCE OF UNMARRIED PARTNER OF HOUSEHOLDER BY HOUSEHOLD TYPE FOR CHILDREN UNDER 18 YEARS IN HOUSEHOLDSPopulation under 18 years in households (excluding householders, spouses, and unmarried partners)TotalNo unmarried partner of householder presentIn family householdsIn female householder no husband present family",34
7527,B09010_006,"RECEIPT OF SUPPLEMENTAL SECURITY INCOME (SSI), CASH PUBLIC ASSISTANCE INCOME, OR FOOD STAMPS/SNAP IN THE PAST 12 MONTHS BY HOUSEHOLD TYPE FOR CHILDREN UNDER 18 YEARS IN HOUSEHOLDSPopulation under 18 years in householdsTotalLiving in household with Supplemental Security Income (SSI) cash public assistance income or Food Stamps/SNAP in the past 12 monthsIn family householdsIn female householder no husband present family",34


# Unemployment

In [77]:
# IV: [population by race; owner-occupied units; MFI; hs grad pct; 
### female hoh pct; unemployment pct; poverty pct; median home value;
### median home year built

metadata_df[(metadata_df['sequence_number'] >= 75) & (metadata_df['description'].apply(lambda x: 'unemploy' in x.lower()))]

ind_vars += ['B23025_005']

# Poverty status

In [81]:
# IV: [population by race; owner-occupied units; MFI; hs grad pct; 
### female hoh pct; unemployment pct; poverty pct; median home value;
### median home year built

metadata_df[(metadata_df['sequence_number'] >= 48) & (metadata_df['description'].apply(lambda x: 'poverty' in x.lower()))]

ind_vars += ['B17001_002']


# Education

In [98]:
# IV: [population by race; owner-occupied units; MFI; hs grad pct; 
### female hoh pct; unemployment pct; poverty pct; median home value;
### median home year built

metadata_df[
    (metadata_df['sequence_number'] >= 43) & 
    (metadata_df['description'].apply(lambda x: 'educational attainment' in x.lower())) & 
    (metadata_df['description'].apply(lambda x: 'high school' in x.lower())) & 
    (metadata_df['description'].apply(lambda x: 'graduate' in x.lower())) 
]

metadata_df[
    (metadata_df['sequence_number'] == 43) & 
    (metadata_df.index >= 9559)
]

ind_vars += ['B15002_011', 'B15002_028']

# Housing, house value, year built

In [107]:
# IV: [population by race; owner-occupied units; MFI; hs grad pct; 
### female hoh pct; unemployment pct; poverty pct; median home value;
### median home year built

metadata_df[
    (metadata_df['sequence_number'] >= 103) & 
    #(metadata_df['description'].apply(lambda x: 'home' in x.lower())) & 
    (metadata_df['description'].apply(lambda x: 'year' in x.lower())) & 
    (metadata_df['description'].apply(lambda x: 'built' in x.lower())) 
]
ind_vars += ['B25003_002', 'B25077_001', 'B25035_001']

In [155]:
ind_vars = ['B02001_001', 'B02001_002', 'B02001_003', 'B03002_012']   # race
ind_vars += ['B19113_001', 'B19113A_001', 'B19113B_001']   # median family income
ind_vars += ['B11001_001', 'B11001_006']   # female householder
ind_vars += ['B23025_005']   # unemployment
ind_vars += ['B17001_002']   # poverty status
ind_vars += ['B15002_011', 'B15002_028']   # high school graduate (male and female separately)
ind_vars += ['B25003_002', 'B25077_001', 'B25035_001'] # housing: owner-occupied units, median value, median year built

metadata_df[metadata_df['variable'].isin(ind_vars)]

,variable,description,sequence_number
varnum,,,
367,B02001_001,RACE% Total population%Total,4
368,B02001_002,RACE%Total population%Total%White alone,4
369,B02001_003,RACE%Total population%Total%Black or African American alone,4
625,B03002_012,HISPANIC OR LATINO ORIGIN BY RACE%Total population%Total%Hispanic or Latino,5
7816,B11001_001,HOUSEHOLD TYPE (INCLUDING LIVING ALONE)% Households%Total,36
7821,B11001_006,HOUSEHOLD TYPE (INCLUDING LIVING ALONE)%Households%Total%Family households%Other family%Female householder no husband present,36
9571,B15002_011,SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER%Population 25 years and over%Total%Male%High school graduate (includes equivalency),43
9588,B15002_028,SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 YEARS AND OVER%Population 25 years and over%Total%Female%High school graduate (includes equivalency),43
10885,B17001_002,POVERTY STATUS IN THE PAST 12 MONTHS BY SEX BY AGE%Population for whom poverty status is determined%Total%Income in the past 12 months below poverty level,48


# Get only the independent variables we need

In [175]:
s = time.time()
estimates_allvars_df = pandas.read_csv(basedir+'estimates_baltimore_city_allvars.csv', low_memory=False, index_col='GEOID')
needcolumns = ind_vars
if ('Geography Name.15' not in needcolumns):
    needcolumns.append('Geography Name.15')
#data_df
estimates_df = estimates_allvars_df[needcolumns]
estimates_df = estimates_df.rename(columns={'Geography Name.15': 'Geography Name'})
estimates_df = estimates_df.assign(tract = pandas.to_numeric(estimates_df['Geography Name'].apply(lambda x: x[re.search('Census Tract ', x).end():re.search(',', x).start()]), errors='coerce'))
estimates_df = estimates_df.set_index('tract')
estimates_df = estimates_df.rename(columns = {'B02001_001': 'total_population', 'B02001_002': 'total_white', 'B02001_003': 'total_black', 'B03002_012': 'total_hispanic'})
estimates_df = estimates_df.rename(columns = {'B19113_001': 'mfi', 'B19113A_001': 'mfi_white', 'B19113B_001': 'mfi_black'})
estimates_df = estimates_df.rename(columns = {'B11001_001': 'total_households', 'B11001_006': 'households_female_holder', 'B23025_005': 'unemployed', 'B17001_002': 'poverty'})
estimates_df = estimates_df.rename(columns = {'B15002_011': 'male_hs_graduates', 'B15002_028': 'female_hs_graduates'})
estimates_df = estimates_df.rename(columns = {'B25003_002': 'total_owner_occupied_housing', 'B25077_001': 'median_home_value', 'B25035_001': 'median_home_year_built'})

estimates_df = estimates_df.assign(total_hs_graduates = estimates_df['male_hs_graduates'] + estimates_df['female_hs_graduates'])

estimates_df = estimates_df.assign(pct_white = estimates_df['total_white'] / estimates_df['total_population'])
estimates_df = estimates_df.assign(pct_black = estimates_df['total_black'] / estimates_df['total_population'])
estimates_df = estimates_df.assign(pct_hispanic = estimates_df['total_hispanic'] / estimates_df['total_population'])
estimates_df = estimates_df.assign(pct_female_hoh = estimates_df['households_female_holder'] / estimates_df['total_households'])
estimates_df = estimates_df.assign(pct_unemployed = estimates_df['unemployed'] / estimates_df['total_population'])
estimates_df = estimates_df.assign(pct_in_poverty = estimates_df['poverty'] / estimates_df['total_population'])
estimates_df = estimates_df.assign(pct_hs_graduates = estimates_df['total_hs_graduates'] / estimates_df['total_population'])
estimates_df = estimates_df.assign(pct_owner_occupied = estimates_df['total_owner_occupied_housing'] / estimates_df['total_households'])

#estimates_df.sample(1).T

cra_iv_columns = ['total_population', 'total_households', 'pct_white', 'pct_black', 'pct_hispanic', 'pct_female_hoh']
cra_iv_columns += ['pct_unemployed', 'pct_in_poverty', 'pct_owner_occupied', 'mfi', 'mfi_white', 'mfi_black']
cra_iv_columns += ['median_home_value', 'median_home_year_built']
cra_iv_data = estimates_df[cra_iv_columns]
cra_iv_data.to_csv(basedir+'cra_iv_estimates.csv', encoding='utf-8')
e = time.time()

print('Done getting needed data in {0:,.0f} seconds!'.format(e-s))

Done getting needed data in 4 seconds!
